In [1]:
import gymnasium as gym


env = gym.make("CartPole-v1")
state = env.reset()

In [2]:
print("Initial state: ", state)

Initial state:  (array([ 0.01126267, -0.02810955,  0.00930572,  0.02807731], dtype=float32), {})


In [3]:
import tensorflow as tf
from tensorflow.keras import layers

# Define the neural network model
def create_model(input_shape, num_actions):
    model = tf.keras.Sequential([
        layers.InputLayer(shape=input_shape),
        layers.Dense(128, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_actions, activation='linear')  # Outputs Q-values for each action
    ])
    return model

2025-01-24 18:49:54.744757: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-24 18:49:54.748304: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-24 18:49:54.757526: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737744594.772208   38412 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737744594.776585   38412 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-24 18:49:54.794674: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [ ]:
input_shape = env.observation_space.shape
num_actions = env.action_space.n
model = create_model(input_shape, num_actions)

model.summary()

2025-01-24 18:49:58.523482: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,410 (68.01 KB)

 Trainable params: 17,410 (68.01 KB)

 Non-trainable params: 0 (0.00 B)

In [5]:
import time
import numpy as np

sample_state = np.random.random((1, 4)).astype(np.float32)

start_time = time.time()
for _ in range(100):
    _ = model.predict(sample_state, verbose=0)
end_time = time.time()

print(f"Average inference time: {(end_time - start_time) / 100:.6f} seconds")

Average inference time: 0.052179 seconds


In [6]:
from collections import deque


learning_rate = 0.0002
gamma = 0.95  # Discount factor for future rewards
epsilon = 1.0  # Initial exploration rate
epsilon_min = 0.1  # Final exploration rate
epsilon_decay = 0.999
batch_size = 32
memory_size = 5000
num_episodes = 500
max_steps_per_episode = 500

replay_memory = deque(maxlen=memory_size)
target_model = create_model(input_shape, num_actions)
target_model.set_weights(model.get_weights())

In [7]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='mse')

#Target model is not compiled because it's weights are not getting updated during training

In [8]:
# Epsilon-greedy policy for action selection
def choose_action(state, epsilon):
    if np.random.rand() < epsilon:
        return np.random.choice(num_actions)  # Explore
    q_values = model.predict(state[np.newaxis], verbose=0)
    return np.argmax(q_values[0])  # Exploit

In [9]:
import random

# Train the model using a batch of experiences
def train_model():
    if len(replay_memory) < batch_size:
        return  # Wait until enough experiences are stored

    batch = random.sample(replay_memory, batch_size)
    states, actions, rewards, next_states, dones = map(np.array, zip(*batch))

    # Compute target Q-values
    next_q_values = target_model.predict(next_states, verbose=0)
    max_next_q_values = np.max(next_q_values, axis=1)
    target_q_values = rewards + gamma * max_next_q_values * (1 - dones)

    # Update the model
    q_values = model.predict(states, verbose=0)
    for i, action in enumerate(actions):
        q_values[i, action] = target_q_values[i]

    model.fit(states, q_values, epochs=1, verbose=0)


In [ ]:
# Training loop
for episode in range(num_episodes):
    state, _ = env.reset()
    done = False
    total_reward = 0

    for step in range(max_steps_per_episode):
        action = choose_action(state, epsilon)  # Choose action
        next_state, reward, terminated, truncated, _ = env.step(action)
        
        done = terminated or truncated

        # Store the experience
        replay_memory.append((state, action, reward, next_state, done))

        # Train the model
        train_model()

        state = next_state
        total_reward += reward

        if done:
            break

    # Update epsilon
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

    # Sync weights with the target model periodically
    if episode % 20 == 0:
        target_model.set_weights(model.get_weights())

    if total_reward >= 400:
        model.save(f"../model/dqn-model-{episode}.keras")
    
    print(f"Episode {episode + 1}: Total Reward = {total_reward}")


env.close()

Episode 1: Total Reward = 12.0
Episode 2: Total Reward = 13.0
Episode 3: Total Reward = 17.0
Episode 4: Total Reward = 22.0
Episode 5: Total Reward = 31.0
Episode 6: Total Reward = 15.0
Episode 7: Total Reward = 14.0
Episode 8: Total Reward = 10.0
Episode 9: Total Reward = 15.0
Episode 10: Total Reward = 14.0
Episode 11: Total Reward = 14.0
Episode 12: Total Reward = 15.0
Episode 13: Total Reward = 22.0
Episode 14: Total Reward = 22.0
Episode 15: Total Reward = 20.0
Episode 16: Total Reward = 15.0
Episode 17: Total Reward = 25.0
Episode 18: Total Reward = 24.0
Episode 19: Total Reward = 34.0
Episode 20: Total Reward = 11.0
Episode 21: Total Reward = 14.0
Episode 22: Total Reward = 14.0
Episode 23: Total Reward = 15.0
Episode 24: Total Reward = 18.0
Episode 25: Total Reward = 22.0
Episode 26: Total Reward = 17.0
Episode 27: Total Reward = 16.0
Episode 28: Total Reward = 17.0
Episode 29: Total Reward = 21.0
Episode 30: Total Reward = 32.0
Episode 31: Total Reward = 26.0
Episode 32: Total

In [12]:
model.save("../model/dqn-model.keras")